### In this notebook we use tensorflow CNN model to find waldo.

In [16]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt


In [17]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    '../data/src1/256-bw',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(64, 64),
    batch_size=32)

Found 317 files belonging to 2 classes.
Using 254 files for training.
Using 63 files for validation.


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

model.fit(train_ds, epochs=10)

model.save('model.h5')



model.evaluate(test_ds)


Epoch 1/10
8/8 [==============================] - 0s 3ms/step - loss: 749.8121 - accuracy: 0.7126
Epoch 2/10


2024-06-08 15:21:58.924026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [254]
	 [[{{node Placeholder/_4}}]]
2024-06-08 15:21:58.924244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [254]
	 [[{{node Placeholder/_4}}]]


8/8 [==============================] - 0s 3ms/step - loss: 150.4830 - accuracy: 0.9016
Epoch 3/10
8/8 [==============================] - 0s 5ms/step - loss: 54.8606 - accuracy: 0.8189
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 44.1353 - accuracy: 0.8425
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 50.0723 - accuracy: 0.7992
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 51.6662 - accuracy: 0.8110
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 28.2604 - accuracy: 0.8543
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 12.2128 - accuracy: 0.8622
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 12.2715 - accuracy: 0.8465
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 34.6070 - accuracy: 0.9048


2024-06-08 15:21:59.596696: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [63]
	 [[{{node Placeholder/_4}}]]
2024-06-08 15:21:59.596855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [63]
	 [[{{node Placeholder/_4}}]]


[34.607025146484375, 0.9047619104385376]